In [1]:
# import libraries
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns 

# for making the output constant across all run
np.random.seed(42)

# display settings & code formatting
pd.options.display.max_columns = 999
%matplotlib inline
%load_ext nb_black

# project paths
project_root_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

data_path = os.path.join(project_root_dir, "data")
os.makedirs(data_path, exist_ok=True)

image_path = os.path.join(project_root_dir, "images")
os.makedirs(image_path, exist_ok=True)

# function for loading data
def load_data(filename, date_col=None, data_path=data_path):
    csv_path = os.path.join(data_path, filename)
    return pd.read_csv(csv_path, parse_dates=date_col)

# function for saving data as csv file
def save_dataframe(df, filename, file_path=data_path):
    path = os.path.join(file_path, filename)
    df.to_csv(path, index=False)


<IPython.core.display.Javascript object>

In [2]:
train = load_data("train.csv", date_col=["Date"])
test = load_data("test.csv", date_col=["Date"])
stores = load_data("stores.csv")
features = load_data("features.csv", date_col=["Date"])
sample_submission = load_data("sampleSubmission.csv")

<IPython.core.display.Javascript object>

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 13.3 MB


<IPython.core.display.Javascript object>

In [4]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


<IPython.core.display.Javascript object>

In [5]:
train.shape

(421570, 5)

<IPython.core.display.Javascript object>

In [6]:
test.shape

(115064, 4)

<IPython.core.display.Javascript object>

## Merge Data

In [7]:
# Merge the stores data with train and test
train = pd.merge(train, stores, how="left", on="Store")
test = pd.merge(test, stores, how="left", on="Store")

# Merge the features data with train and test
train = pd.merge(train, features, how="left", on=["Store", "Date"])
test = pd.merge(test, features, how="left", on=["Store", "Date"])

<IPython.core.display.Javascript object>

In [8]:
train.isnull().sum() / len(train) * 100

Store            0.000000
Dept             0.000000
Date             0.000000
Weekly_Sales     0.000000
IsHoliday_x      0.000000
Type             0.000000
Size             0.000000
Temperature      0.000000
Fuel_Price       0.000000
MarkDown1       64.257181
MarkDown2       73.611025
MarkDown3       67.480845
MarkDown4       67.984676
MarkDown5       64.079038
CPI              0.000000
Unemployment     0.000000
IsHoliday_y      0.000000
dtype: float64

<IPython.core.display.Javascript object>

In [9]:
test.isnull().sum() / len(test) * 100

Store            0.000000
Dept             0.000000
Date             0.000000
IsHoliday_x      0.000000
Type             0.000000
Size             0.000000
Temperature      0.000000
Fuel_Price       0.000000
MarkDown1        0.129493
MarkDown2       24.879198
MarkDown3        8.542203
MarkDown4       11.200723
MarkDown5        0.000000
CPI             33.165890
Unemployment    33.165890
IsHoliday_y      0.000000
dtype: float64

<IPython.core.display.Javascript object>

In [10]:
train.columns

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday_x', 'Type', 'Size',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y'],
      dtype='object')

<IPython.core.display.Javascript object>

In [11]:
train.drop(["IsHoliday_y"], axis=1, inplace=True)
test.drop(["IsHoliday_y"], axis=1, inplace=True)

# rename column
train.rename(columns={"IsHoliday_x": "IsHoliday"}, inplace=True)
test.rename(columns={"IsHoliday_x": "IsHoliday"}, inplace=True)

<IPython.core.display.Javascript object>

## Feature Engineering

In [12]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,2010-02-05,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,2010-02-12,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,2010-02-19,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,2010-02-26,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,2010-03-05,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


<IPython.core.display.Javascript object>

In [13]:
train["Date"].dt.weekday

0         4
1         4
2         4
3         4
4         4
         ..
421565    4
421566    4
421567    4
421568    4
421569    4
Name: Date, Length: 421570, dtype: int64

<IPython.core.display.Javascript object>

In [14]:
## Datetime features
train["Year"] = train["Date"].dt.year
train["Month"] = train["Date"].dt.month
train["Day"] = train["Date"].dt.day
train["WeekOfYear"] = train["Date"].dt.weekofyear
train["DayOfWeek"] = train["Date"].dt.dayofweek
train["Weekend"] = (train["Date"].dt.weekday >= 5).astype(int)

<ipython-input-14-827adf94b2da>:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  train["WeekOfYear"] = train["Date"].dt.weekofyear


<IPython.core.display.Javascript object>

In [15]:
test["Year"] = test["Date"].dt.year
test["Month"] = test["Date"].dt.month
test["Day"] = test["Date"].dt.day
test["WeekOfYear"] = test["Date"].dt.weekofyear
test["DayOfWeek"] = test["Date"].dt.dayofweek
test["Weekend"] = (test["Date"].dt.weekday >= 5).astype(int)

<ipython-input-15-9e957ccb6653>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  test["WeekOfYear"] = test["Date"].dt.weekofyear


<IPython.core.display.Javascript object>

In [16]:
# convert cat cols to num
train["IsHoliday"] = train["IsHoliday"].map({True: 1, False: 0})
test["IsHoliday"] = test["IsHoliday"].map({True: 1, False: 0})
train["Type"] = train["Type"].map({"A": 3, "B": 2, "C": 1})
test["Type"] = test["Type"].map({"A": 3, "B": 2, "C": 1})

<IPython.core.display.Javascript object>

In [17]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Day,WeekOfYear,DayOfWeek,Weekend
0,1,1,2010-02-05,24924.50,0,3,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,2010,2,5,5,4,0
1,1,1,2010-02-12,46039.49,1,3,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,2010,2,12,6,4,0
2,1,1,2010-02-19,41595.55,0,3,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,2010,2,19,7,4,0
3,1,1,2010-02-26,19403.54,0,3,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,2010,2,26,8,4,0
4,1,1,2010-03-05,21827.90,0,3,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,2010,3,5,9,4,0


<IPython.core.display.Javascript object>

## Data Preparation 

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder

<IPython.core.display.Javascript object>

In [19]:
test.head()

,Store,Dept,Date,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Day,WeekOfYear,DayOfWeek,Weekend
0,1,1,2012-11-02,0,3,151315,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,2012,11,2,44,4,0
1,1,1,2012-11-09,0,3,151315,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,2012,11,9,45,4,0
2,1,1,2012-11-16,0,3,151315,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,2012,11,16,46,4,0
3,1,1,2012-11-23,1,3,151315,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,2012,11,23,47,4,0
4,1,1,2012-11-30,0,3,151315,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,2012,11,30,48,4,0


<IPython.core.display.Javascript object>

In [20]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Year,Month,Day,WeekOfYear,DayOfWeek,Weekend
0,1,1,2010-02-05,24924.50,0,3,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,2010,2,5,5,4,0
1,1,1,2010-02-12,46039.49,1,3,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,2010,2,12,6,4,0
2,1,1,2010-02-19,41595.55,0,3,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,2010,2,19,7,4,0
3,1,1,2010-02-26,19403.54,0,3,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,2010,2,26,8,4,0
4,1,1,2010-03-05,21827.90,0,3,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,2010,3,5,9,4,0


<IPython.core.display.Javascript object>

In [21]:
# features and labels of train and test set
# labels of test are not provided as we need to predict them

X_train = train.drop(["Weekly_Sales"], axis=1).copy()
y_train = train["Weekly_Sales"].copy()

X_test = test.copy()

# drop and save the date column in a variable
train_date = X_train.pop("Date")
test_date = X_test.pop("Date")


#### Data preparation pipeline

# select numerical and categorical columns
num_cols = X_train.select_dtypes(exclude=["object"]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

# numerical date preprocessing pipeline
num_pipe = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

# categorical data preprocessing pipeline
cat_pipe = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"),
    OneHotEncoder(handle_unknown="ignore", sparse=False),
)

# full pipeline
full_pipe = ColumnTransformer(
    [("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)]
)

full_pipe

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Store', 'Dept', 'IsHoliday', 'Type', 'Size',
                                  'Temperature', 'Fuel_Price', 'MarkDown1',
                                  'MarkDown2', 'MarkDown3', 'MarkDown4',
                                  'MarkDown5', 'CPI', 'Unemployment', 'Year',
                                  'Month', 'Day', 'WeekOfYear', 'DayOfWeek',
                                  'Weekend']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='NA',
                                                  

<IPython.core.display.Javascript object>

# Build Models

## Evaluation Metric

The evaluation metric for this task is weighted mean absolute error. But this metric is not available in scikit-learn. So, we have to create this metric using scikit-learn's `make_scorer` function. 

In [22]:
from sklearn.metrics import make_scorer


def WMAE(y, y_pred):
    weights = X_train["IsHoliday"].apply(lambda x: 1 if x == 0 else 5)
    return np.round(np.sum(weights * abs(y - y_pred)) / (np.sum(weights)), 2)


wmae_scorer = make_scorer(WMAE, greater_is_better=False)

<IPython.core.display.Javascript object>

In [23]:
# def WMAE(dataset, real, predicted):
#     """
#     Implementation of competition evaluation metric
#     """
#     weights = dataset.IsHoliday.apply(lambda x: 5 if x else 1)
#     return np.round(np.sum(weights * abs(real - predicted)) / (np.sum(weights)), 2)

<IPython.core.display.Javascript object>

In [24]:
# sample_weight = X_train["IsHoliday"].apply(lambda x: 1 if x == 0 else 5)
# sample_weight_frame = pd.DataFrame(sample_weight, index=X_train.index)


# from sklearn.metrics import make_scorer


# def WMAE_b(y_test, y_pred):
#     y_pred_df = pd.DataFrame(y_pred, index=y_test.index)

#     weights_5 = (
#         sample_weight_frame.loc[(y_test.index)]
#         .loc[sample_weight_frame.IsHoliday == 5]
#         .index
#     )
#     weights_1 = (
#         sample_weight_frame.loc[(y_test.index)]
#         .loc[sample_weight_frame.IsHoliday == 1]
#         .index
#     )

#     sum_5 = np.sum(
#         5 * (abs(y_test.loc[weights_5].values - y_pred_df.loc[weights_5].values))
#     )
#     sum_1 = np.sum(abs(y_test.loc[weights_1].values - y_pred_df.loc[weights_1].values))

#     return np.round((sum_5 + sum_1) / (5 * len(weights_5) + len(weights_1)), 2)


# my_score = make_scorer(WMAE, greater_is_better=False)

<IPython.core.display.Javascript object>

## Linear Regression

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_reg = make_pipeline(full_pipe, LinearRegression())
wmae_scores = cross_val_score(lin_reg, X_train, y_train, cv=5, scoring=wmae_scorer)

print("-----------")
print("wmae scores:", -wmae_scores)
print("-----------")
print("Mean wmae: ", np.mean(-wmae_scores))
print("-----------")

-----------
wmae scores: [3050.24 3383.37 3178.66 2949.65 2356.8 ]
-----------
Mean wmae:  2983.744
-----------


<IPython.core.display.Javascript object>

## Support Vector Machines

In [26]:
from sklearn.svm import LinearSVR

lin_svm = make_pipeline(full_pipe, LinearSVR(random_state=42))
wmae_scores = cross_val_score(lin_svm, X_train, y_train, cv=5, scoring=wmae_scorer)

print("-----------")
print("wmae scores:", -wmae_scores)
print("-----------")
print("Mean wmae: ", np.mean(-wmae_scores))
print("-----------")

-----------
wmae scores: [2683.4  2986.28 2767.58 2526.72 2312.86]
-----------
Mean wmae:  2655.368
-----------


<IPython.core.display.Javascript object>

## Decision Trees

In [27]:
from sklearn.tree import DecisionTreeRegressor

dtree = make_pipeline(full_pipe, DecisionTreeRegressor(random_state=42))
wmae_scores = cross_val_score(dtree, X_train, y_train, cv=5, scoring=wmae_scorer)

print("-----------")
print("wmae scores:", -wmae_scores)
print("-----------")
print("Mean wmae: ", np.mean(-wmae_scores))
print("-----------")

-----------
wmae scores: [2173.63 1639.07 1464.34 1605.88 1052.81]
-----------
Mean wmae:  1587.146
-----------


<IPython.core.display.Javascript object>

## Random Forest 

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf = make_pipeline(full_pipe, RandomForestRegressor(random_state=42))
wmae_scores = cross_val_score(rf, X_train, y_train, cv=5, scoring=wmae_scorer)

print("-----------")
print("wmae scores:", -wmae_scores)
print("-----------")
print("Mean wmae: ", np.mean(-wmae_scores))
print("-----------")

-----------
wmae scores: [2019.03 1501.11 1325.04 1473.69  928.27]
-----------
Mean wmae:  1449.4280000000003
-----------


<IPython.core.display.Javascript object>

## XGBoost

In [29]:
from xgboost import XGBRegressor

xgb = make_pipeline(full_pipe, XGBRegressor(random_state=42))
wmae_scores = cross_val_score(xgb, X_train, y_train, cv=5, scoring=wmae_scorer)

print("-----------")
print("wmae scores:", -wmae_scores)
print("-----------")
print("Mean wmae: ", np.mean(-wmae_scores))
print("-----------")

-----------
wmae scores: [1762.87 1465.51 1466.73 1189.6  1197.38]
-----------
Mean wmae:  1416.4180000000001
-----------


<IPython.core.display.Javascript object>

The weighted mean absolute error for linear model is around 3000 but tree based methods reduced this error by half. So we will focus more on random forest and xgboost moddels. For now, let's make a submission and see our score on
kaggle leaderboard. 

## Submission

In [30]:
sample_submission.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0


<IPython.core.display.Javascript object>

In [31]:
from xgboost import XGBRegressor

# train the model on whole data and make predictions
xgb = make_pipeline(full_pipe, XGBRegressor(random_state=42))
xgb.fit(X_train, y_train)
xgb_y_pred = xgb.predict(X_test)

<IPython.core.display.Javascript object>

In [32]:
sample_submission["Weekly_Sales"] = xgb_y_pred

<IPython.core.display.Javascript object>

In [34]:
save_dataframe(sample_submission, "xgb_default_sub.csv")

<IPython.core.display.Javascript object>

The weighted mean absolute error we got on the leaderboard is much higher. It's around 5445. Let's also use the random forest model and see how this perform.

In [35]:
from sklearn.ensemble import RandomForestRegressor

rf = make_pipeline(full_pipe, RandomForestRegressor(random_state=42))
rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
sample_submission["Weekly_Sales"] = rf_y_pred
save_dataframe(sample_submission, "rf_default_sub.csv")

<IPython.core.display.Javascript object>

The wmae of the random forest model is around 3550, which is far less than the xgboost model. Later we will improve 
this rf model to get better performance. 